In [1]:
import os
import random
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import ElectraModel, ElectraTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

## Load BERT and Tokenizer

In [2]:
bert = ElectraModel.from_pretrained('monologg/koelectra-base-v3-discriminator')
tokenizer = ElectraTokenizer.from_pretrained('tokenizer')

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Label Dicts

In [3]:
labels = ['[PAD]', 'E_B', 'E_I', 'O']
num_labels = len(labels)
id2label = {k: v for k, v in enumerate(labels)}
label2id = {v: k for k, v in id2label.items()}

## Load Data and Preprocess for Training

In [4]:
data = pd.read_pickle('data/preprocessed.pkl')

In [5]:
data['tokens'] = data.tokens.apply(lambda x: ['[CLS]'] + x + ['[SEP]'])
data['labels'] = data.labels.apply(lambda x: ['O'] + x + ['O'])

In [6]:
data.tokens.apply(lambda x: len(x)).max()

233

In [7]:
max_len = 256

In [8]:
data['tokens'] = data.tokens.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))
data['labels'] = data.labels.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))

In [9]:
tokens_lst = data.tokens.to_list()
labels_lst = data.labels.to_list()

In [10]:
X_train, X_eval, y_train, y_eval = train_test_split(tokens_lst, 
                                                    labels_lst, 
                                                    test_size=0.2, shuffle=True, random_state=42)

In [11]:
train_data = []
for tokens, labels in zip(X_train, y_train):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)

    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    train_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])

In [12]:
eval_data = []
for tokens, labels in zip(X_eval, y_eval):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)
    
    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    eval_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])

In [13]:
# idx = random.randrange(0, len(train_data) - 1)
# for x, xm, y in zip(train_data[idx][0], train_data[idx][1], train_data[idx][2]):
#     print(x, xm, y)

# idx = random.randrange(0, len(eval_data) - 1)
# for x, xm, y in zip(eval_data[idx][0], eval_data[idx][1], eval_data[idx][2]):
#     print(x, xm, y)

## HP Config

In [ ]:
batch_size = 128
LEARNING_RATE = 5e-5
N_EPOCHS = 30

## Create Dataset and Generate Dataloader

In [14]:
class TaggerDataset(Dataset): 
    def __init__(self, data):
        self.data = data
    
    def __len__(self): 
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = self.data[idx][0]
        mask = self.data[idx][1]
        label_ids = self.data[idx][2]
        return (torch.LongTensor(input_ids), torch.LongTensor(mask), torch.LongTensor(label_ids))

In [15]:
train_dataset = TaggerDataset(train_data)
eval_dataset = TaggerDataset(eval_data)

In [16]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
eval_loader = DataLoader(eval_dataset, batch_size = batch_size, shuffle = True)

## Instantiate Model

In [17]:
class BERTPoSTagger(nn.Module):
    def __init__(self,
                 bert,
                 output_dim, 
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        self.dropout = nn.Dropout(dropout)
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text, mask):
        embedded = self.dropout(self.bert(text, mask)[0])
        predictions = self.fc(self.dropout(embedded))
        return predictions

In [18]:
OUTPUT_DIM = num_labels
DROPOUT = 0.25

model = BERTPoSTagger(bert,
                      OUTPUT_DIM, 
                      DROPOUT)
model.bert.resize_token_embeddings(len(tokenizer))

Embedding(36223, 768)

In [20]:
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = 0)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=0, factor=0.7, min_lr=0)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NGPU = torch.cuda.device_count()
if NGPU > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(NGPU)))
    # model = torch.nn.DataParallel(model, device_ids=[0,1])
    # torch.multiprocessing.set_start_method('spawn', force=True)
model = model.to(device)
criterion = criterion.to(device)

In [22]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])

    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

In [23]:
def categorical_f1(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    max_preds_no_pad = max_preds[non_pad_elements].squeeze(1).detach().cpu()
    y_no_pad = y[non_pad_elements].detach().cpu()
    
    f1_macro = f1_score(y_no_pad, max_preds_no_pad, average='macro')
    f1_micro = f1_score(y_no_pad, max_preds_no_pad, average='micro')    
    
    return f1_macro, f1_micro

In [24]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    predictions_set = None
    tags_set = None
    for batch in iterator:
        text = batch[0].to(device)
        mask = batch[1].to(device)
        tags = batch[2].to(device)

        predictions = model(text, mask)
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        if predictions_set == None:
            predictions_set = predictions
            tags_set = tags
        else:
            predictions_set = torch.cat([predictions_set, predictions], dim=0)
            tags_set = torch.cat([tags_set, tags], dim=0)
        
        loss = criterion(predictions, tags)
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    f1_macro, f1_micro = categorical_f1(predictions_set, tags_set, tag_pad_idx)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1_macro, f1_micro

In [25]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    predictions_set = None
    tags_set = None
    with torch.no_grad():
        for batch in iterator:
            text = batch[0].to(device)
            mask = batch[1].to(device)
            tags = batch[2].to(device)
            
            predictions = model(text, mask)
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            if predictions_set == None:
                predictions_set = predictions
                tags_set = tags
            else:
                predictions_set = torch.cat([predictions_set, predictions], dim=0)
                tags_set = torch.cat([tags_set, tags], dim=0)
            
            loss = criterion(predictions, tags)
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        f1_macro, f1_micro = categorical_f1(predictions_set, tags_set, tag_pad_idx)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1_macro, f1_micro

In [26]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_f1_mac, train_f1_mic = train(model, train_loader, optimizer, criterion, 0)
    valid_loss, valid_acc, valid_f1_mac, valid_f1_mic = evaluate(model, eval_loader, criterion, 0)
    
    cur_lr = scheduler.optimizer.state_dict()['param_groups'][0]['lr']
    scheduler.step(valid_loss)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'event_tagger.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'Learning Rate: {cur_lr}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%', end=' | ')
    print(f'Train F1 Mac: {train_f1_mac*100:.2f}% | Train F1 Mic: {train_f1_mic*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc:.2f}%', end=' |  ')
    print(f'Val. F1 Mac: {valid_f1_mac*100:.2f}% |  Val. F1 Mic: {valid_f1_mic*100:.2f}%', end='\n\n')

Epoch: 01 | Epoch Time: 1m 34s
Learning Rate: 5e-05
	Train Loss: 0.856 | Train Acc: 0.69% | Train F1 Mac: 23.66% | Train F1 Mic: 68.35%
	 Val. Loss: 0.646 |  Val. Acc: 0.78% |  Val. F1 Mac: 29.14% |  Val. F1 Mic: 77.65%

Epoch: 02 | Epoch Time: 1m 13s
Learning Rate: 5e-05
	Train Loss: 0.622 | Train Acc: 0.77% | Train F1 Mac: 34.76% | Train F1 Mic: 77.15%
	 Val. Loss: 0.474 |  Val. Acc: 0.81% |  Val. F1 Mac: 43.61% |  Val. F1 Mic: 81.14%

Epoch: 03 | Epoch Time: 1m 12s
Learning Rate: 5e-05
	Train Loss: 0.427 | Train Acc: 0.83% | Train F1 Mac: 53.95% | Train F1 Mic: 82.63%
	 Val. Loss: 0.320 |  Val. Acc: 0.87% |  Val. F1 Mac: 72.27% |  Val. F1 Mic: 87.42%

Epoch: 04 | Epoch Time: 0m 43s
Learning Rate: 5e-05
	Train Loss: 0.284 | Train Acc: 0.89% | Train F1 Mac: 76.42% | Train F1 Mic: 88.91%
	 Val. Loss: 0.236 |  Val. Acc: 0.91% |  Val. F1 Mac: 82.32% |  Val. F1 Mic: 91.30%

Epoch: 05 | Epoch Time: 0m 42s
Learning Rate: 5e-05
	Train Loss: 0.215 | Train Acc: 0.92% | Train F1 Mac: 83.75% | T

KeyboardInterrupt: 